# Notebook to download the data for SEISMO-VRE  
Except where otherwise noted, this Jupyter Notebook is made available under the GPL-3.0 license. For details, please see the `LICENSE` file in the main directory.
# Python Version

## Data list
### Analyses workflow 
1. [Atmospheric data from NASA](#Atmosphere-data)
2. [Ionosphere Swarm data from ESA](#Ionosphere-data)
3. [Geomagnetic indexes (Dst and ap)](#Geomagnetic-indexes)
4. [North geomagnetic pole](#North-geomagnetic-pole).

### Reconfigurable parameters
**Please, remember to update the global variables according to the earthquake you want to analyse in the main notebook**

In [2]:
#Global variables initialization

from cdflib import CDF, cdfepoch
from datetime import datetime, timedelta
import glob
import io
import math
import numpy as np
import os
import pandas as pd
import requests
import scipy.io
from scipy.io import savemat
import sys
import time
from typing import Union
import viresclient
from viresclient import set_token
from viresclient import SwarmRequest
import xarray as xr
import csv
from tqdm import tqdm
from scipy.io import savemat # Import scipy for .mat file saving


# Earthquake origin time
EQ_year = 2016
EQ_month = 8
EQ_day = 24
EQ_hour = 1
EQ_minute = 36
EQ_second = 32

# Epicenter latitude and longitude
EPILAT = 42.6980
EPILON = 13.2340

# Mainshock magnitude and depth
EQ_mag = 6.5
EQ_depth = 10

# Fault strike
# Here is inserted a mean strike of the two main faults: Amatrice 155 and Norcia 151.
# This strike must be a positive value.
Fault_strike = 153

# Time windows
day_before = 8 * 30  # example,  8*30 days before
day_after = 0


## Atmosphere data

The atmospheric analysis uses the MERRA2 climatological archive data from NASA (doi: <a href="https://doi.org/10.5067/KLICLTZ8EM9D"> 10.5067/KLICLTZ8EM9D</a>).  

**If you need to change the case study, you must delete the file `Atmospheric_data_from_MERRA2.mat`**. Otherwise, the analysis could potentially mix data from different events.

In this session the data from MERRA-2 Climatological archive will be downloaded and analysed
**Important note:**  
The MERRA-2 data are open but require a free account on EarthData Nasa portal. The following line will ask you your username (generally your registration email) and your password in order to download the data.

In order to mask the password the Matlab tool logindlg has been used. Proper credit to: Jeremy Smith (2025). login (https://www.mathworks.com/matlabcentral/fileexchange/8499-login), MATLAB Central File Exchange. Retrieved January 25, 2025.

In [ ]:
# --- CONSTANTS AND CONFIGURATION ---
DATA_DIR = 'data'
OUTPUT_FILENAME = os.path.join(DATA_DIR, 'Atmospheric_data_from_MERRA2_local.csv')
MATLAB_OUTPUT_FILENAME = os.path.join(DATA_DIR, 'Atmospheric_data_from_MERRA2.mat') # MATLAB output filename

# NASA Earthdata credentials (NOTE: These are placeholder credentials and should be handled securely)
NASA_USERNAME = "username" # YOU NEED TO INSERT YOUR EO-NASA USERNAME HERE
NASA_PASSWORD = "password" # YOU NEED TO INSERT YOUR EO-NASA PASSWORD HERE

# MERRA2 products to download
MERRA_PRODUCTS = {
    'inst1_2d_int_Nx': {'url_shortname': 'M2I1NXINT', 'variables': {'TQV': 'TCWV'}}, # Total Column Water Vapor
    'tavg1_2d_aer_Nx': {'url_shortname': 'M2T1NXAER', 'variables': {'DMSSMASS': 'DMS', 'SO2SMASS': 'SO2', 'TOTEXTTAU': 'AOT'}}, # Aerosols: DMS, SO2, Aerosol Optical Thickness
    'tavg1_2d_chm_Nx': {'url_shortname': 'M2T1NXCHM', 'variables': {'COSC': 'CO', 'TO3': 'TO3'}}, # Chemistry: Carbon Monoxide, Ozone
    'inst1_2d_lfo_Nx': {'url_shortname': 'M2I1NXLFO', 'variables': {'QLML': 'HUM', 'TLML': 'SAT'}}, # Low Frequency: Humidity, Surface Air Temperature
    'tavg1_2d_flx_Nx': {'url_shortname': 'M2T1NXFLX', 'variables': {'EFLUX': 'SLHF', 'PRECTOT': 'RAIN'}} # Fluxes: Surface Latent Heat Flux, Total Precipitation
}

MAX_RETRIES = 6
RETRY_DELAYS_SECONDS = [10, 30, 60, 180, 300, 600]

# --- FUNCTIONS ---

def datetime_to_matlab_datenum(dt: datetime) -> float:
    """
    Converts a Python datetime.datetime object to the MATLAB datenum format.
    
    MATLAB datenum is the number of days since 0000-01-00 (MATLAB epoch).
    Python's ordinal date is the number of days since 0001-01-01 (Python epoch).
    A 366-day correction is applied to account for the epoch difference.
    """
    if not isinstance(dt, datetime):
        return np.nan # Handle invalid or missing values

    # 1. Calculate integer days (integer part of datenum)
    # The correction of 366 days accounts for the epoch shift (MATLAB year 0000 vs Python year 0001)
    datenum_days = dt.toordinal() + 366

    # 2. Return the full datenum (integer days + fraction)
    return float(datenum_days)


def get_merra_version(date_obj: datetime) -> str:
    """Returns the correct MERRA2 data version based on the date."""
    if date_obj < datetime(1992, 1, 1): return 'MERRA2_100'
    if date_obj < datetime(2001, 1, 1): return 'MERRA2_200'
    if date_obj < datetime(2011, 1, 1): return 'MERRA2_300'
    if (datetime(2020, 9, 1) <= date_obj <= datetime(2020, 9, 30)) or \
       (datetime(2021, 6, 1) <= date_obj <= datetime(2021, 9, 30)):
        return 'MERRA2_401'
    return 'MERRA2_400'

def download_merra_file(url: str, local_path: str) -> bool:
    """Downloads the MERRA2 file with authentication and retry logic."""
    for attempt in range(MAX_RETRIES):
        try:
            with requests.get(url, auth=(NASA_USERNAME, NASA_PASSWORD), stream=True, timeout=60) as r:
                if r.status_code == 200:
                    with open(local_path, 'wb') as f:
                        for chunk in r.iter_content(chunk_size=8192):
                            f.write(chunk)
                    return True
                else:
                    print(f"HTTP {r.status_code} for {url}")
        except Exception as e:
            print(f"Attempt {attempt+1}/{MAX_RETRIES} failed for {url}. Error: {e}")
        if attempt < MAX_RETRIES - 1:
            delay = RETRY_DELAYS_SECONDS[attempt]
            print(f"Retrying in {delay} seconds...")
            time.sleep(delay)
    print(f"WARNING: Failed to download {url}")
    return False

def process_merra_day(date_obj: datetime, bbox: dict, ut_hour: int) -> dict:
    """
    Downloads, reads, processes (subsets and averages), and safely deletes 
    a single day's MERRA2 file. Returns the aggregated data.
    """
    year, month, day = date_obj.year, date_obj.month, date_obj.day
    version_f = get_merra_version(date_obj)
    
    # The reference time for extraction is the day's date + ut_hour + 30 minutes
    time_target_dt = datetime(date_obj.year, date_obj.month, date_obj.day, ut_hour, 30)
    
    # Store the complete datetime object. It will be converted to string for CSV 
    # and to datenum for MATLAB later.
    daily_data = {"date": time_target_dt}
    base_url = "https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/hyrax"

    for product_name, config in MERRA_PRODUCTS.items():
        # Construct the remote OPeNDAP URL
        file_path = (f"MERRA2/{config['url_shortname']}.5.12.4/{year}/{month:02d}/"
                     f"{version_f}.{product_name}.{year}{month:02d}{day:02d}.nc4.nc4?")
        opendap_url = f"{base_url}/{file_path}"
        # Define the local temporary file path
        local_file = os.path.join(DATA_DIR, f"{version_f}.{product_name}.{year}{month:02d}{day:02d}.nc4")

        # Download the file
        if not download_merra_file(opendap_url, local_file):
            # If download fails, fill with NaN and continue to next product
            for df_col in config['variables'].values():
                daily_data[df_col] = np.nan
            continue

        # Read and process data safely
        try:
            # Open the downloaded NetCDF file with xarray
            ds = xr.open_dataset(local_file, engine="netcdf4", decode_times=True)
            try:
                # Define geographical and time slicing parameters
                lat_slice = slice(bbox['min_lat'], bbox['max_lat'])
                lon_slice = slice(bbox['min_lon'], bbox['max_lon'])
                time_target = np.datetime64(time_target_dt)

                for nc_var, df_col in config['variables'].items():
                    if nc_var not in ds:
                        # Variable missing in the file
                        daily_data[df_col] = np.nan
                        continue
                    
                    # Spatial subsetting
                    spatial_subset = ds[nc_var].sel(lat=lat_slice, lon=lon_slice)
                    
                    # Temporal subsetting if 'time' dimension exists
                    if 'time' in spatial_subset.dims:
                        # Find the closest time step to the target time
                        data_subset = spatial_subset.sel(time=time_target, method='nearest')
                    else:
                        # For variables without a time dimension
                        data_subset = spatial_subset
                        
                    # Calculate the spatial mean and store the value
                    daily_data[df_col] = float(data_subset.mean().values)
            finally:
                ds.close()  # Explicitly close the file before deletion
        except Exception as e:
            print(f"Error reading {local_file}: {e}")
            # Fill with NaN if processing fails
            for df_col in config['variables'].values():
                if df_col not in daily_data:
                    daily_data[df_col] = np.nan
        finally:
            # Delete the local file after processing
            if os.path.exists(local_file):
                try:
                    os.remove(local_file)
                except PermissionError:
                    print(f"Cannot delete {local_file}, file still in use. Manual cleanup needed.")
                    pass

    return daily_data


# --- FUNZIONE save_to_matlab MODIFICATA PER IL NUOVO FORMATO ---
def save_to_matlab(df: pd.DataFrame, filename: str):
    """
    Saves the pandas DataFrame to a MATLAB .mat file.
    It structures the output as: {Time_Var1, Var1, Time_Var2, Var2, ...}
    """
    mat_dict = {}
    
    # Prepare the datenum converter
    datenum_converter = np.vectorize(datetime_to_matlab_datenum)
    
    # 1. Convert the array of datetime objects (from the 'date' column) to datenum once
    # This array will be used for all variables
    time_datenum_array = datenum_converter(df['date'].values)
    
    # 2. Iterate through all data columns and create the paired Time_VarX variables
    # The 'date' column contains the original datetime objects and should not be saved as data
    data_columns = [col for col in df.columns if col != 'date']
    
    for col in data_columns:
        # Save the datenum array first, using a custom name (e.g., Time_TCWV)
        time_var_name = f"Time_{col}"
        mat_dict[time_var_name] = time_datenum_array
        
        # Save the data array next
        mat_dict[col] = df[col].values.astype(float) 

    # Save the dictionary to a .mat file (v5 format)
    savemat(filename, mat_dict)
    print(f"\nData successfully saved to MATLAB format: {filename}")


# --- MAIN EXECUTION ---
def main():
    # 1. Setup
    os.makedirs(DATA_DIR, exist_ok=True)
    
    # 2. Calculate Bounding Box (BBox) and Target Time
    # Calculate radius according to Dobrovolsky's formula (in degrees)
    radius_dobrovolsky = 10 ** (0.43 * EQ_mag) / 111.0
    # Calculate lat/lon half-radii for a square BBox (using sqrt(2) for simplification)
    lat_radius, lon_radius = radius_dobrovolsky / np.sqrt(2), radius_dobrovolsky / np.sqrt(2)
    bbox = {
        "min_lat": EPILAT - lat_radius,
        "max_lat": EPILAT + lat_radius,
        "min_lon": EPILON - lon_radius,
        "max_lon": EPILON + lon_radius
    }

    # Convert longitude to UT hour (Local Midnight Approximation)
    local_midnight = -EPILON / 15.0 # Time difference from UTC in hours (15 degrees/hour)
    ut_hour = int(round(local_midnight)) % 24 # Round to nearest hour and ensure it's within [0, 23]

    # 3. Generate Dates to Analyze
    start_year = 1980
    end_year = datetime.now().year - 1
    days_to_analyze = []
    # Loop through all years from start_year to end_year
    for year in range(start_year, end_year + 1):
        center_date = datetime(year, EQ_month, EQ_day) # The 'earthquake' date for that year
        # Define the window around the 'earthquake' date
        start_date, end_date = center_date - timedelta(days=day_before), center_date + timedelta(days=day_after)
        # Generate and add dates within the window
        # pd.date_range ensures standard datetime objects are used
        days_to_analyze.extend(pd.date_range(start_date, end_date))
    # Remove duplicates (in case windows overlap) and sort
    days_to_analyze = sorted(list(set(days_to_analyze)))

    # 4. Prepare CSV Output
    # The 'date' column in the CSV will store the full time as a string.
    all_columns = ["date"] + [col for cfg in MERRA_PRODUCTS.values() for col in cfg['variables'].values()]

    # Read existing dates to skip already processed days
    existing_dates = set()
    if os.path.isfile(OUTPUT_FILENAME):
        try:
            # Read the existing CSV for the update phase
            df_check = pd.read_csv(OUTPUT_FILENAME)
            # Use string format (Y-m-d H:M:S) for comparison
            existing_dates = set(df_check["date"].astype(str))
        except:
            print(f"WARNING: Could not read existing dates from {OUTPUT_FILENAME}. Processing all days.")
            pass
            
    # 5. Main Processing Loop
    pbar = tqdm(total=len(days_to_analyze), desc="Processing days")
    new_data_rows = [] # List to collect new data before appending

    for day_obj in days_to_analyze:
        # The reference string for the check must include hour and minutes
        day_str_check = datetime(day_obj.year, day_obj.month, day_obj.day, ut_hour, 30).strftime("%Y-%m-%d %H:%M:%S")
        
        # Skip if already processed
        if day_str_check in existing_dates:
            pbar.update(1)
            continue

        # Process the day (download, subset, average)
        daily_results = process_merra_day(day_obj, bbox, ut_hour)
        
        # Convert the full datetime object to a standard string for CSV saving
        daily_results['date'] = daily_results['date'].strftime("%Y-%m-%d %H:%M:%S") 
        new_data_rows.append(daily_results)

        pbar.update(1)

    pbar.close()
    
    # 5b. Write only the new data
    if new_data_rows:
        # If the file exists, append. Otherwise, write the header.
        write_header = not os.path.isfile(OUTPUT_FILENAME)
        df_new = pd.DataFrame(new_data_rows, columns=all_columns)
        # Append new data to the CSV file
        df_new.to_csv(OUTPUT_FILENAME, mode='a', header=write_header, index=False)
        print(f"\nAppended {len(new_data_rows)} new data rows to {OUTPUT_FILENAME}")
    else:
        print("\nProcessing complete. No new data to append.")


    # 6. Save to MATLAB .mat file
    # Read the final, consolidated CSV data into a DataFrame
    try:
        # Read the complete CSV file, converting the 'date' string column back to datetime objects
        final_df = pd.read_csv(OUTPUT_FILENAME, parse_dates=['date']) 
        save_to_matlab(final_df, MATLAB_OUTPUT_FILENAME)
        
        # Delete the temporary CSV file
        if os.path.exists(OUTPUT_FILENAME):
            os.remove(OUTPUT_FILENAME)
            print(f"Successfully deleted temporary CSV file: {OUTPUT_FILENAME}")
            
    except Exception as e:
        print(f"FATAL ERROR: Could not read CSV file to save to MATLAB format. Error: {e}")

if __name__ == "__main__":
    main()

Processing days:   0%|          | 0/10845 [00:00<?, ?it/s]

HTTP 404 for https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/hyrax/MERRA2/M2I1NXINT.5.12.4/1979/12/MERRA2_100.inst1_2d_int_Nx.19791228.nc4.nc4?
Retrying in 10 seconds...
HTTP 404 for https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/hyrax/MERRA2/M2I1NXINT.5.12.4/1979/12/MERRA2_100.inst1_2d_int_Nx.19791228.nc4.nc4?
Retrying in 30 seconds...
HTTP 404 for https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/hyrax/MERRA2/M2I1NXINT.5.12.4/1979/12/MERRA2_100.inst1_2d_int_Nx.19791228.nc4.nc4?
Retrying in 60 seconds...
HTTP 404 for https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/hyrax/MERRA2/M2I1NXINT.5.12.4/1979/12/MERRA2_100.inst1_2d_int_Nx.19791228.nc4.nc4?
Retrying in 180 seconds...
HTTP 404 for https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/hyrax/MERRA2/M2I1NXINT.5.12.4/1979/12/MERRA2_100.inst1_2d_int_Nx.19791228.nc4.nc4?
Retrying in 300 seconds...
HTTP 404 for https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/hyrax/MERRA2/M2I1NXINT.5.12.4/1979/12/MERRA2_100.inst1_2d_int_Nx.19791228.nc4.nc4

Processing days:   0%|          | 1/10845 [49:38<8972:43:43, 2978.77s/it]

HTTP 404 for https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/hyrax/MERRA2/M2T1NXFLX.5.12.4/1979/12/MERRA2_100.tavg1_2d_flx_Nx.19791228.nc4.nc4?
HTTP 404 for https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/hyrax/MERRA2/M2I1NXINT.5.12.4/1979/12/MERRA2_100.inst1_2d_int_Nx.19791229.nc4.nc4?
Retrying in 10 seconds...
HTTP 404 for https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/hyrax/MERRA2/M2I1NXINT.5.12.4/1979/12/MERRA2_100.inst1_2d_int_Nx.19791229.nc4.nc4?
Retrying in 30 seconds...
HTTP 404 for https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/hyrax/MERRA2/M2I1NXINT.5.12.4/1979/12/MERRA2_100.inst1_2d_int_Nx.19791229.nc4.nc4?
Retrying in 60 seconds...
HTTP 404 for https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/hyrax/MERRA2/M2I1NXINT.5.12.4/1979/12/MERRA2_100.inst1_2d_int_Nx.19791229.nc4.nc4?
Retrying in 180 seconds...
HTTP 404 for https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/hyrax/MERRA2/M2I1NXINT.5.12.4/1979/12/MERRA2_100.inst1_2d_int_Nx.19791229.nc4.nc4?
Retrying in 300 seconds..

Processing days:   0%|          | 2/10845 [1:39:13<8965:21:10, 2976.60s/it]

...

Processing days:   0%|          | 4/10845 [3:18:33<8968:12:26, 2978.10s/it]

HTTP 404 for https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/hyrax/MERRA2/M2T1NXFLX.5.12.4/1979/12/MERRA2_100.tavg1_2d_flx_Nx.19791231.nc4.nc4?


Processing days:   5%|▍         | 489/10845 [27:48:21<609:54:27, 212.02s/it]

Attempt 1/6 failed for https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/hyrax/MERRA2/M2T1NXFLX.5.12.4/1982/01/MERRA2_100.tavg1_2d_flx_Nx.19820103.nc4.nc4?. Error: HTTPSConnectionPool(host='goldsmr4.gesdisc.eosdis.nasa.gov', port=443): Read timed out. (read timeout=60)
Retrying in 10 seconds...
Attempt 2/6 failed for https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/hyrax/MERRA2/M2T1NXFLX.5.12.4/1982/01/MERRA2_100.tavg1_2d_flx_Nx.19820103.nc4.nc4?. Error: HTTPSConnectionPool(host='goldsmr4.gesdisc.eosdis.nasa.gov', port=443): Max retries exceeded with url: /opendap/hyrax/MERRA2/M2T1NXFLX.5.12.4/1982/01/MERRA2_100.tavg1_2d_flx_Nx.19820103.nc4.nc4 (Caused by NewConnectionError("<urllib3.connection.HTTPSConnection object at 0x000001C7F4F49D90>: Failed to establish a new connection: [WinError 10060] Impossibile stabilire la connessione. Risposta non corretta della parte connessa dopo l'intervallo di tempo oppure mancata risposta dall'host collegato"))
Retrying in 30 seconds...


Processing days:  24%|██▍       | 2590/10845 [143:04:33<469:29:49, 204.75s/it]

## Ionosphere data
In this session, the magnetic data from Swarm three-identical satellites are pre-processed.

Data must be downloaded from ESA's ftp dissemination service, which requires a free EO user account (you can create one here: https://eoiam-idp.eo.esa.int/) and placed in one folder of your computer were the VRE is running.

In the following example the data are inside the folder **ndir** = `E:\Swarm_data`, but **remember to replace with your own data path**  
The ESA Swarm file names appear as:  
SW_OPER_MAGA_LR_1B_20131125T110252_20131125T235959_0602_MDR_MAG_LR.cdf  
SW_OPER_MAGA_LR_1B_20201027T134456_20201027T150128_0605_MDR_MAG_LR.cdf  
...  
SW_OPER_MAGC_LR_1B_20140524T000000_20140524T235959_0602_MDR_MAG_LR.cdf


The first part of the following code selects the data in the time range defined for the analysis of the earthquake.


In [ ]:
# --- 1. User Configuration & Parameters ---

# Set your Vires token here.
# You will be prompted to enter the token in the console the first time.
# Information on how to get a token: https://viresclient.readthedocs.io/en/latest/access_token.html
viresclient.set_token("https://vires.services/ows")

# --- 2. Calculation of Time and Space Windows ---

# Build the main datetime object for the earthquake
eq_time =datetime(EQ_year, EQ_month, EQ_day, EQ_hour, EQ_minute, EQ_second)

# Define the start and end times for the data request
time_start = eq_time - timedelta(days=day_before)
time_end = eq_time + timedelta(days=day_after)

# Calculate the preparatory radius based on Dobrovolsky's formula (in degrees)
# This defines the primary area of interest around the epicenter.
radius_dobrovolsky = 10**(0.43 * EQ_mag) / 111

# Define a broader longitude bounding box for the initial data request
# We add a buffer to ensure we capture complete satellite tracks passing near the area.
lon_buffer = 5.0
min_lon = EPILON - radius_dobrovolsky - lon_buffer
max_lon = EPILON + radius_dobrovolsky + lon_buffer

print(f"Requesting data from {time_start.strftime('%Y-%m-%d')} to {time_end.strftime('%Y-%m-%d')}")
print(f"Spatial filter (longitude): {min_lon:.2f} to {max_lon:.2f}")


def dat_orden_alpha(x, SAT):
    Mmlat = 70  # Maximum latitude
    sat_map = {'A': 1, 'B': 2, 'C': 3, 'S': 4}
    sat = sat_map.get(SAT, 0)
    # Filter by latitude (column 7 → index 6 zero‑based)
    y = x[np.abs(x[:, 6]) <= Mmlat]
    nn = y.shape[0]
    if nn <= 1:
        return np.empty((0, x.shape[1] + 2))
    yn = np.zeros((nn - 1, x.shape[1] + 2))  # Two additional columns: sat and track number
    m = 1  # track number
    for k in range(nn - 1):
        row = np.concatenate([[sat], y[k, 0:6], [m], y[k, 6:]])  # Two extra columns: sat and track number
        yn[k, :] = row
        # Track‑change condition
        if abs(y[k, 6] - y[k + 1, 6]) > 0.5 or abs(y[k, 7] - y[k + 1, 7]) > 1:
            m += 1
    return yn

# --- 3. Viresclient Data Request ---

print("Connecting to Vires server and downloading data...")
request = viresclient.SwarmRequest()

# Set the collection for all three Swarm satellites (A, B, C)
request.set_collection("SW_OPER_MAGA_LR_1B", "SW_OPER_MAGB_LR_1B", "SW_OPER_MAGC_LR_1B")

# Define the specific data products to fetch
request.set_products(
    measurements=["B_NEC", "F", "Flags_F", "Flags_B", "Flags_q", "Flags_Platform"],
    sampling_step="PT1S" # Resample to 1-second intervals
)

# Apply the spatial filter on the server to reduce download size
request.set_range_filter('Longitude', min_lon, max_lon)

# Execute the data fetching for the specified time interval
data = request.get_between(
    start_time=time_start,
    end_time=time_end
)

# Load the downloaded data into an xarray.Dataset for easy manipulation
ds = data.as_xarray()
print("Download complete.")


# --- 4. Setup Output Directory ---
output_dir = os.path.join('data', 'Swarm_tracks')
os.makedirs(output_dir, exist_ok=True)
print(f"Output files will be saved in: {output_dir}")


# --- 5. Data Processing & Saving ---
print("Processing downloaded data and saving tracks...")
total_files_saved = 0

# The downloaded dataset contains data from all satellites mixed together.
# We group the data by the 'Spacecraft' identifier ('A', 'B', 'C') to process each one.
for sat_id, ds_sat in ds.groupby('Spacecraft'):
    print(f"  - Processing tracks for Swarm satellite {sat_id}")

    # Check if this satellite group has any data
    if ds_sat.sizes['Timestamp'] == 0:
        print(f"  - No data for satellite {sat_id} in the given window.")
        continue

    # Convert timestamp to a pandas DatetimeIndex for easy component extraction
    times = pd.to_datetime(ds_sat['Timestamp'].values)

    # Reconstruct the numpy array format required by the 'dat_orden_alpha' function
    # This structure matches the one expected from the original CDF file reading script.
    # Columns: 0-5:Time, 6:Lat, 7:Lon, 8:Radius, 9-12:Flags, 13-15:B_NEC, 16:F
    n = len(times)
    x = np.empty((n, 17), dtype=float)
    x[:, 0] = times.year
    x[:, 1] = times.month
    x[:, 2] = times.day
    x[:, 3] = times.hour
    x[:, 4] = times.minute
    x[:, 5] = times.second + times.microsecond / 1e6
    x[:, 6] = ds_sat['Latitude'].values
    x[:, 7] = ds_sat['Longitude'].values
    x[:, 8] = ds_sat['Radius'].values
    x[:, 9] = ds_sat['Flags_F'].values
    x[:, 10] = ds_sat['Flags_B'].values
    x[:, 11] = ds_sat['Flags_q'].values
    x[:, 12] = ds_sat['Flags_Platform'].values
    x[:, 13:16] = ds_sat['B_NEC'].values
    x[:, 16] = ds_sat['F'].values

    # Identify and number the satellite tracks
    yn = dat_orden_alpha(x, sat_id)

    # Apply a final, more precise spatial filter based on the Dobrovolsky radius.
    # A generous 15-degree buffer is added.
    # The longitude is now in column 9 of the 'yn' array.
    mask = np.abs(yn[:, 9] - EPILON) <= (radius_dobrovolsky + 15.0)
    y_sel = yn[mask]

    if y_sel.shape[0] > 0:
        print(f"  - Found {y_sel.shape[0]} data points for satellite {sat_id} passing the filter.")

        # Group data by day and save one file per day.
        # The date columns are at indices 1 (Year), 2 (Month), 3 (Day) in the processed 'yn' array.
        # Create a unique integer for each date (e.g., 20160824) to group the data rows.
        dates_int = (y_sel[:, 1] * 10000 + y_sel[:, 2] * 100 + y_sel[:, 3]).astype(int)
        unique_dates = np.unique(dates_int)

        print(f"  - Grouping data into {len(unique_dates)} daily files for satellite {sat_id}.")

        for date_val in unique_dates:
            # Select all data points for the current day
            day_mask = (dates_int == date_val)
            current_day_data = y_sel[day_mask]

            # Count the number of unique tracks in the current day's data
            # The track number is in the 8th column (index 8)
            num_tracks_in_file = len(np.unique(current_day_data[:, 8]))

            # Extract year, month, day from the integer value for the filename
            year = date_val // 10000
            month = (date_val % 10000) // 100
            day = date_val % 100

            # Format the filename to include the date
            filename = f"Swarm_tracks_{year:04d}{month:02d}{day:02d}_{sat_id}.mat"
            filepath = os.path.join(output_dir, filename)

            # Save the current day's track data and the track count
            savemat(filepath, {
                "y": current_day_data
            })
            print(f"    - Saved {filename} containing {num_tracks_in_file} track(s).")
            total_files_saved += 1
    else:
        print(f"  - No tracks passed the final spatial filter for satellite {sat_id}.")

print(f"\nProcessing complete. Total files saved: {total_files_saved}")

Setting access token for https://vires.services/ows... Generate a token at https://vires.services/accounts/tokens/

Token saved for https://vires.services/ows
Requesting data from 2015-12-28 to 2016-08-24
Spatial filter (longitude): 2.61 to 23.85
Connecting to Vires server and downloading data...


          |  [ Elapsed: 00:00, Remaining: ?]  

Processing:    0%|          |  [ Elapsed: 00:00, Remaining: ? ] [1/5] 

Downloading:   0%|          |  [ Elapsed: 00:00, Remaining: ? ] (53.362MB)

...

Downloading:   0%|          |  [ Elapsed: 00:00, Remaining: ? ] (52.982MB)

Processing:    0%|          |  [ Elapsed: 00:00, Remaining: ? ] [4/5] 

Downloading:   0%|          |  [ Elapsed: 00:00, Remaining: ? ] (53.729MB)

Processing:    0%|          |  [ Elapsed: 00:00, Remaining: ? ] [5/5] 

Downloading:   0%|          |  [ Elapsed: 00:00, Remaining: ? ] (42.995MB)

## Geomagnetic indexes
The following cells are necessary to update the geomagnetic indexes Dst and ap files.  
*Please, note that is not necessary to update these files for new case studies except it requires more recent data not previously stored in the same files.  
### Updating the Dst geomagnetic index

In [4]:
import numpy as np
import requests
import datetime
import sys


def parse_dst_data_from_html(html_content):
    """
    Extracts Dst data from the raw HTML content of the web page.

    Args:
        html_content (str): The text content of the HTML page.

    Returns:
        list: A list of lists, where each inner list represents a row of numeric data.
              Returns an empty list if the data cannot be found.
    """
    try:
        # Find the start of the data table by searching for the "DAY" header
        start_index = html_content.find('DAY')
        if start_index == -1:
            return []

        # Find the end of the data table (usually before the </pre> tag)
        end_index = html_content.find('</pre>', start_index)
        if end_index == -1:
            return []

        # Extract the text block containing only the data
        data_block = html_content[start_index:end_index]
        # Split the block into lines, skipping the header
        lines = data_block.strip().split('\n')[1:]
        
        # Define the specific character slices for each of the 24 hourly values
        HOURLY_SLICES = [
            (3, 7), (7, 11), (11, 15), (15, 19), (19, 23),
            (23, 27), (27, 31), (31, 35), (36, 40), (40, 44), (44, 48),
            (48, 52), (52, 56), (56, 60), (60, 64), (64, 68), (69, 73),
            (73, 77), (77, 81), (81, 85), (85, 89), (89, 93), (93, 97), (97, 101)
        ]
        
        parsed_data = []
        for line in lines:
            if not line.strip():
                continue

            try:
                # 1. Isolate the day number (first 2 chars) and the hourly data string
                day_str = line[0:3].strip()
                hourly_data_str = line[:]

                # 2. Initialize the 'parts' list with the day number
                parts = [int(day_str)]

                # 3. Loop through the *specific* slices, not a regular pattern
                for start, end in HOURLY_SLICES:
                    chunk = hourly_data_str[start:end]
                    value = int(chunk.strip())
                    parts.append(value)

                # 4. A valid row should have 1 day + 24 hours = 25 values
                if len(parts) == 25:
                    parsed_data.append(parts)
                else:
                    print(f"Warning: Skipped line for day {day_str}. Parsed {len(parts) - 1}/24 values.")

            except (ValueError, IndexError) as e:
                # print(f"Warning: Skipped line due to a data format error: {line} -> Error: {e}")
                continue

    except Exception as e:
        print(f"Unexpected error during HTML parsing: {e}")
        return []

    return parsed_data

def aggiorna_dst_to_final():
    """
    Downloads and updates the Dst (Disturbance storm time) index data
    from the World Data Center for Geomagnetism, Kyoto.
    """
    output_filename = 'data/dst_index2.dat'

    # 1. Load existing data
    try:
        idst_old = np.loadtxt(output_filename, dtype=int)
        print(f"Loaded {idst_old.shape[0]} existing records from '{output_filename}'.")
    except FileNotFoundError:
        print(f"File '{output_filename}' not found. Starting with an empty dataset.")
        # Create an empty array with the correct number of columns (27: year, month, day, 24 hourly values)
        idst_old = np.empty((0, 27), dtype=int)
    except Exception as e:
        print(f"Error reading file '{output_filename}': {e}")
        return

    # Get the current year for the loop's upper limit
    current_year = datetime.datetime.now().year

    # The data types to download, in order of priority (final overwrites provisional, etc.)
    data_sources = ['dst_final', 'dst_provisional', 'dst_realtime']
    base_url = 'https://wdc.kugi.kyoto-u.ac.jp'

    # 2. Loop to download and update data
    for data_type in data_sources:
        print(f"\n--- Starting processing for data type: {data_type} ---")
        for year in range(2000, current_year + 1):
            # Show a progress indicator
            sys.stdout.write(f"Processing Year: {year}\r")
            sys.stdout.flush()

            for month in range(1, 13):
                # Build the URL
                url = f"{base_url}/{data_type}/{year}{month:02d}/index.html"
                
                try:
                    # Perform the web request
                    response = requests.get(url, timeout=10)
                    # Check if the request was successful
                    if response.status_code == 200:
                        # Parse data from the HTML
                        monthly_data = parse_dst_data_from_html(response.text)

                        if not monthly_data:
                            continue

                        # Update the numpy array with the new data
                        for day_data in monthly_data:
                            day = day_data[0]
                            # Create the complete row with year and month
                            new_row = np.array([year, month] + day_data, dtype=int)

                            # Check if a row for this date already exists
                            existing_row_index = np.where(
                                (idst_old[:, 0] == year) &
                                (idst_old[:, 1] == month) &
                                (idst_old[:, 2] == day)
                            )[0]

                            if existing_row_index.size == 0:
                                # The row does not exist, so add it
                                idst_old = np.vstack([idst_old, new_row])
                            else:
                                # The row exists, update it if it's different
                                idx = existing_row_index[0]
                                if not np.array_equal(idst_old[idx], new_row):
                                    idst_old[idx] = new_row

                except requests.exceptions.RequestException:
                    # If the URL is not found (e.g., for future months), ignore the error
                    pass

    print("\nDownload and update completed.")

    # 3. Filter and sort the data
    if idst_old.shape[0] > 0:
        # Remove rows where any of the hourly values is 9999
        # The hourly data columns range from index 3 to the end
        original_rows = idst_old.shape[0]
        idst_old = idst_old[np.all(idst_old[:, 3:] != 9999, axis=1)]
        rows_removed = original_rows - idst_old.shape[0]
        if rows_removed > 0:
            print(f"Removed {rows_removed} rows containing 9999 values.")

        # Sort the array by year, month, and day
        # np.lexsort sorts using keys from last to first
        indices = np.lexsort((idst_old[:, 2], idst_old[:, 1], idst_old[:, 0]))
        idst_old = idst_old[indices]
        print("Data sorted by date.")

    # 4. Save the updated data to the file
    try:
        # The '%d' format ensures all numbers are written as integers
        np.savetxt(output_filename, idst_old, fmt='%d', delimiter=' ')
        print(f"Final data successfully saved to '{output_filename}'. Total records: {idst_old.shape[0]}")
    except Exception as e:
        print(f"Error while saving the file '{output_filename}': {e}")


if __name__ == '__main__':
    aggiorna_dst_to_final()


File 'data/dst_index2.dat' not found. Starting with an empty dataset.

--- Starting processing for data type: dst_final ---


Processing Year: 2025
--- Starting processing for data type: dst_provisional ---
Processing Year: 2025
--- Starting processing for data type: dst_realtime ---
Processing Year: 2025
Download and update completed.
Removed 19 rows containing 9999 values.
Data sorted by date.
Final data successfully saved to 'data/dst_index2.dat'. Total records: 9447


### Updating the ap geomagnetic index
The following cell defines a simple function to convert the kp to ap according to standard tables.

In [7]:
from datetime import datetime
def kp2ap(kp: str) -> Union[int, None]:
    """
    Converts a Kp index string to its corresponding Ap index value.
    This function uses a dictionary for efficient and clean mapping.

    Args:
        kp (str): The Kp index as a string (e.g., "0o", "3-", "5+").

    Returns:
        Union[int, None]: The corresponding Ap index as an integer, or
                          None if the Kp string is not found in the map.
    """
    # A dictionary to map Kp string values to Ap integer values.
    # This is more efficient and readable than a long series of if statements.
    kp_to_ap_map = {
        '0o': 0, '0+': 2, '1-': 3, '1o': 4, '1+': 5,
        '2-': 6, '2o': 7, '2+': 9, '3-': 12, '3o': 15,
        '3+': 18, '4-': 22, '4o': 27, '4+': 32, '5-': 39,
        '5o': 48, '5+': 56, '6-': 67, '6o': 80, '6+': 94,
        '7-': 111, '7o': 132, '7+': 154, '8-': 179, '8o': 207,
        '8+': 236, '9-': 300, '9o': 400
    }
    return kp_to_ap_map.get(kp)


def _parse_gfz_data(data_string: str) -> list:
    """
    Helper function to parse the fixed-width text data from GFZ Potsdam.
    It processes a raw string, extracts data, and converts Kp to Ap values.

    Args:
        data_string (str): The raw text content downloaded from the URL.

    Returns:
        list: A list of lists, where each inner list represents a row of parsed data
              [year, month, day, ap1, ap2, ..., ap8].
    """
    parsed_data = []
    # Split the raw string into individual lines.
    lines = data_string.strip().split('\n')
    for line in lines:
        # Skip header, footer, or empty lines by checking length and first character.
        if len(line) < 33 or not line[0].isdigit():
            continue

        try:
            # Extract date components by slicing the string.
            year = 2000 + int(line[0:2])
            month = int(line[2:4])
            day = int(line[4:6])

            # Extract the 8 Kp values and convert them to Ap values using the helper.
            ap_values = [
                kp2ap(line[8:10]),
                kp2ap(line[11:13]),
                kp2ap(line[14:16]),
                kp2ap(line[17:19]),
                kp2ap(line[21:23]),
                kp2ap(line[24:26]),
                kp2ap(line[27:29]),
                kp2ap(line[30:32]),
            ]

            # If any Kp conversion failed, skip this line to avoid corrupt data.
            if any(ap is None for ap in ap_values):
                print(f"Warning: Skipping line with invalid Kp value: {line}")
                continue

            # Create the full row [year, month, day, ap_values...] and append it.
            row = [year, month, day] + ap_values
            parsed_data.append(row)
        except (ValueError, IndexError):
            # Handle potential errors if a line is malformed.
            print(f"Warning: Skipping malformed line: {line}")
            continue

    return parsed_data


def update_ap(full_update: bool):
    """
    Main function to update the Ap index data file 'ap_index2.dat'.

    Args:
        full_update (bool):
            If True, it performs a complete update, fetching all definitive data
            from a starting year and overwriting the local file.
            If False, it performs an incremental update, fetching only the
            previous and current months and appending new data to the local file.
    """
    output_filename = 'data/ap_index2.dat'
    iap = []

    # --- FULL UPDATE LOGIC ---
    if full_update:
        print("Starting full update...")
        iap = []  # Reset data for a full update.
        start_year = 2000
        now = datetime.now()
        current_year = now.year
        current_month = now.month

        # Calculate the total number of historical months to fetch.
        # This goes up to two months before the current one, as those are considered definitive.
        total_months = (current_year - start_year) * 12 + current_month - 2

        for month_index in range(1, total_months + 1):
            # Calculate the specific year and month to fetch for this iteration.
            year_to_check = start_year + math.floor((month_index - 1) / 12)
            month_to_check = month_index - (year_to_check - start_year) * 12

            # Format the URL for the historical data FTP server.
            year_str = str(year_to_check)
            url = f"http://ftp.gwdg.de/pub/geophys/kp-ap/tab/kp{year_str[2:]}{month_to_check:02d}.tab"

            print(f"Fetching historical data: {url}")
            try:
                # Fetch data from the URL.
                response = requests.get(url, timeout=10)
                response.raise_for_status()  # Raise an exception for HTTP errors (4xx or 5xx).

                # Parse the downloaded text and extend the main data list.
                ap_table = _parse_gfz_data(response.text)
                if ap_table:
                    iap.extend(ap_table)
            except requests.exceptions.RequestException as e:
                print(f"Error fetching {url}: {e}")

    # --- INCREMENTAL UPDATE LOGIC ---
    else:
        print("Starting incremental update...")
        # For incremental updates, we first load the existing local data.
        try:
            # np.loadtxt is a simple way to load space-delimited files.
            iap = np.loadtxt(output_filename, dtype=int).tolist()
        except (IOError, ValueError):  # Handles FileNotFoundError and empty files.
            print(f"Warning: '{output_filename}' not found or empty. Will create a new one.")
            iap = []

    # --- FETCH PREVIOUS AND CURRENT MONTHS (for both update types) ---

    # URL for the previous month's data (preliminary).
    url_prev = 'https://www-app3.gfz-potsdam.de/kp_index/pqlyymm.tab'
    ap_table_prev = []
    print(f"Fetching previous month's data: {url_prev}")
    try:
        response_prev = requests.get(url_prev, timeout=10)
        response_prev.raise_for_status()
        ap_table_prev = _parse_gfz_data(response_prev.text)
    except requests.exceptions.RequestException as e:
        print(f"Error fetching previous month's data: {e}")

    # URL for the current month's data (quicklook).
    url_curr = 'https://www-app3.gfz-potsdam.de/kp_index/qlyymm.tab'
    ap_table_curr = []
    print(f"Fetching current month's data: {url_curr}")
    try:
        response_curr = requests.get(url_curr, timeout=10)
        response_curr.raise_for_status()
        ap_table_curr = _parse_gfz_data(response_curr.text)
    except requests.exceptions.RequestException as e:
        print(f"Error fetching current month's data: {e}")

    # --- COMBINE AND WRITE DATA ---

    if full_update:
        # In a full update, we append the newly fetched preliminary and quicklook data.
        if ap_table_prev:
            iap.extend(ap_table_prev)
        if ap_table_curr:
            iap.extend(ap_table_curr)

        # Overwrite the output file with the complete dataset.
        print(f"Writing complete dataset to '{output_filename}'...")
        with open(output_filename, 'w') as file_output:
            for row in iap:
                # The original script repeats each of the 8 Ap values three times.
                values_to_write = row[:3]
                for val in row[3:]:
                    values_to_write.extend([val, val, val])
                file_output.write(f"{' '.join(map(str, values_to_write))}\n")
    else:
        # In an incremental update, we only append new, unique rows.
        print(f"Appending new data to '{output_filename}'...")
        # Create a set of existing dates for fast lookups to prevent duplicates.
        existing_dates = {(row[0], row[1], row[2]) for row in iap}

        # Combine the previous and current month data.
        new_data = ap_table_prev + ap_table_curr

        rows_to_add = []
        for row in new_data:
            current_date_tuple = (row[0], row[1], row[2])
            if current_date_tuple not in existing_dates:
                rows_to_add.append(row)
                # Add to set to also handle duplicates within the new data itself.
                existing_dates.add(current_date_tuple)

        # Sort data by date before writing to ensure chronological order.
        rows_to_add.sort(key=lambda x: (x[0], x[1], x[2]))

        if rows_to_add:
            # Open the file in append mode to add the new rows.
            with open(output_filename, 'a') as file_output:
                for row in rows_to_add:
                    values_to_write = row[:3]
                    for val in row[3:]:
                        values_to_write.extend([val, val, val])
                    # Add a leading newline for appended entries, just like the original script.
                    # This assumes the file ends with a newline.
                    file_output.write(f"\n{' '.join(map(str, values_to_write))}")
            print(f"Added {len(rows_to_add)} new entries.")
        else:
            print("No new data to add.")

    print("Update process finished.")


# --- EXAMPLE OF HOW TO RUN THE FUNCTION ---
if __name__ == '__main__':
    # To run an incremental update (most common case):
    # This will only add new data to 'ap_index2.dat'.
    update_ap(full_update=True)

    # To run a full update (takes longer and overwrites the file):
    # This will rebuild 'ap_index2.dat' from the year 2000.
    # update_ap(full_update=True)

Starting full update...
Fetching historical data: http://ftp.gwdg.de/pub/geophys/kp-ap/tab/kp0001.tab
Fetching historical data: http://ftp.gwdg.de/pub/geophys/kp-ap/tab/kp0002.tab
Fetching historical data: http://ftp.gwdg.de/pub/geophys/kp-ap/tab/kp0003.tab
Fetching historical data: http://ftp.gwdg.de/pub/geophys/kp-ap/tab/kp0004.tab
Fetching historical data: http://ftp.gwdg.de/pub/geophys/kp-ap/tab/kp0005.tab
Fetching historical data: http://ftp.gwdg.de/pub/geophys/kp-ap/tab/kp0006.tab
Fetching historical data: http://ftp.gwdg.de/pub/geophys/kp-ap/tab/kp0007.tab
Fetching historical data: http://ftp.gwdg.de/pub/geophys/kp-ap/tab/kp0008.tab
Fetching historical data: http://ftp.gwdg.de/pub/geophys/kp-ap/tab/kp0009.tab
Fetching historical data: http://ftp.gwdg.de/pub/geophys/kp-ap/tab/kp0010.tab
Fetching historical data: http://ftp.gwdg.de/pub/geophys/kp-ap/tab/kp0011.tab
Fetching historical data: http://ftp.gwdg.de/pub/geophys/kp-ap/tab/kp0012.tab
Fetching historical data: http://ftp.gwd

## North geomagnetic pole
The following cell updates the ap geomagnetic index in the file `ap_index2.dat`.

In [8]:
def scrape_geomagnetic_poles(url):
    """
    Scrapes the geomagnetic poles table from a given URL
    using pandas to correctly handle complex headers.

    Args:
        url (str): The URL of the web page to scrape.

    Returns:
        pandas.DataFrame: A pandas DataFrame containing the table data.
                          Returns None if an error occurs.
    """
    try:
        # 1. Send an HTTP request to get the page content
        print(f"Contacting URL: {url}...")
        response = requests.get(url)
        # Check if the request was successful (status code 200)
        response.raise_for_status()
        print("Web page downloaded successfully.")

        # 2. Use pandas.read_html, specifying that the first two rows (0, 1)
        #    are the header. We pass response.text via io.StringIO.
        tables = pd.read_html(io.StringIO(response.text), header=[0, 1])

        if not tables or len(tables) < 2:
            print("Error: The requested table (index 1) was not found on the page.")
            return None

        # 3. The table of interest is the second one found (index 1).
        df = tables[1]

        # 4. The header has multiple levels (MultiIndex). Simplify it.
        new_columns = []
        for col_level1, col_level2 in df.columns:
            # If the first header level contains 'Year', the column is 'Year'.
            if 'Year' in col_level1:
                new_columns.append('Year')
            # Otherwise, join the two levels to create a unique name.
            else:
                # Replace potential newline characters for cleaner names
                col_level1 = col_level1.replace('\n', ' ').strip()
                col_level2 = col_level2.replace('\n', ' ').strip()
                new_columns.append(f"{col_level1} {col_level2}")

        df.columns = new_columns

        # Convert the 'Year' column to string for consistent cleaning
        df['Year'] = df['Year'].astype(str)

        print("Data extracted and converted to DataFrame.")
        return df

    except requests.exceptions.RequestException as e:
        print(f"Error during HTTP request: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None


# URL of the page to scrape data from
URL_GEOMAGNETIC_POLES = "https://wdc.kugi.kyoto-u.ac.jp/poles/polesexp.html"

# Execute the function and get the DataFrame
geomagnetic_data = scrape_geomagnetic_poles(URL_GEOMAGNETIC_POLES)

# If data was extracted successfully, process it
if geomagnetic_data is not None:
    print("\n--- Extracted Geomagnetic Pole Data ---")
    # print(geomagnetic_data.to_string()) # Print omitted for brevity

    # --- SECTION: Process, Interpolate, and save the data to a .dat file ---
    try:
        # 1. Select the relevant columns (ensure correct column names)
        lat_col_name = 'North geomagnetic pole Lat.'
        lon_col_name = 'North geomagnetic pole Long.'
        columns_to_process = ['Year', lat_col_name, lon_col_name]
        data_to_process = geomagnetic_data[columns_to_process].copy()

        # 2. Function to convert coordinates from string to float with the correct sign
        def convert_geo_coord(coord_str):
            """Converts a coordinate string (e.g., '78.7N') into a float number."""
            coord_str = str(coord_str).strip()
            if not coord_str or coord_str.lower() in ('nan', 'none', '-'):
                return None
            
            # Extract the numerical part and the direction (the last character)
            value_str = coord_str[:-1]
            direction = coord_str[-1].upper()

            try:
                value = float(value_str)
                # Apply a negative sign for South (S) and West (W)
                if direction in ['S', 'W']:
                    return -value
                return value
            except (ValueError, TypeError):
                # Returns None if the number conversion fails
                return None

        # Apply the conversion function to the latitude and longitude columns
        data_to_process[lat_col_name] = data_to_process[lat_col_name].apply(convert_geo_coord)
        data_to_process[lon_col_name] = data_to_process[lon_col_name].apply(convert_geo_coord)

        # Remove any rows where the conversion failed in lat or lon columns (resulting in NaN)
        data_to_process.dropna(subset=[lat_col_name, lon_col_name], inplace=True)
        
        # Convert 'Year' to integer for proper time-series indexing
        data_to_process['Year'] = data_to_process['Year'].astype(int)

        # 3. INTERPOLATION LOGIC
        # Set 'Year' as the index for time-series operations
        data_to_process.set_index('Year', inplace=True)
        
        # Create a new index with annual steps (from min year to max year of data)
        min_year = data_to_process.index.min()
        max_year = data_to_process.index.max()
        # Ensure the index covers every year in the range
        new_index = pd.Index(range(min_year, max_year + 1), name='Year')

        # Reindex the DataFrame to insert rows with NaN for missing years
        data_interpolated = data_to_process.reindex(new_index)
        
        # Perform linear interpolation (fill NaN values between known points)
        # 'linear' method is used, treating the index (Year) as the variable to interpolate over.
        data_interpolated.interpolate(method='linear', axis=0, inplace=True)
        
        # Reset the index to have 'Year' as a column again
        data_interpolated.reset_index(inplace=True)

        # Handle potential NaNs at the beginning/end if extrapolation is required (optional, but good practice)
        # If the first or last point is NaN (e.g., if the data started/ended with NaNs that were kept after dropna)
        # we might need to handle them, but for this dataset, they should be filled by reindex/interpolate.
        data_interpolated.dropna(inplace=True)


        # 4. Save the file
        output_dir = 'data'
        os.makedirs(output_dir, exist_ok=True)
        output_filename = os.path.join(output_dir, 'polos_igrf_swarm.dat')

        # Manually write the file to use a multi-character separator (', ') and no header
        with open(output_filename, 'w') as f:
            for index, row in data_interpolated.iterrows():
                # Format Year as integer and Lat/Long as float with 4 decimal places
                line = f"{int(row['Year'])}, {row[lat_col_name]:.4f}, {row[lon_col_name]:.4f}"
                f.write(line + '\n')

        print(f"\nSuccessfully interpolated data (annual step) saved to '{output_filename}'.")
        print(f"The file contains {len(data_interpolated)} records from {min_year} to {max_year}.")

    except KeyError as e:
        print(f"\nError: One of the requested columns was not found. Check the column names. {e}")
    except Exception as e:
        print(f"\nAn error occurred while saving the file: {e}")

Contacting URL: https://wdc.kugi.kyoto-u.ac.jp/poles/polesexp.html...
Web page downloaded successfully.
Data extracted and converted to DataFrame.

--- Extracted Geomagnetic Pole Data ---

Successfully interpolated data (annual step) saved to 'data\polos_igrf_swarm.dat'.
The file contains 131 records from 1900 to 2030.
